In [ ]:
no_of_training_records = 20000 # -1 for all recods in the inputs

In [ ]:
!pip install pandas
import pandas as pd
dataframe = pd.read_csv('/data/CognitiveBanking/data/train_indessa.csv')
dataframe = dataframe.head(no_of_training_records)
dataframe

In [ ]:
import os
import shutil
from os.path import expanduser
modelDir = expanduser("~")+'/model'
tbDir = expanduser("~")+'/tb'

if os.path.exists(modelDir):
    shutil.rmtree(modelDir) 
    os.makedirs(modelDir)
else:
    os.makedirs(modelDir)
print(os.listdir(modelDir))
modelPath = modelDir+'/model.ckpt'


if os.path.exists(tbDir):
    shutil.rmtree(tbDir) 
    os.makedirs(tbDir)
else:
    os.makedirs(tbDir)
tb_logs_path = tbDir

In [ ]:
from sklearn.model_selection import train_test_split
traindf, validatedf = train_test_split(dataframe, test_size=0.01)

In [ ]:
dataframe.loc[:, ("y1")] = dataframe.loc[:, ("loan_status")]
dataframe.loc[:, ("y2")] = dataframe["y1"] == 0           # y2 is the negation of y1
dataframe.loc[:, ("y2")] = dataframe["y2"].astype(int)    # Turn TRUE/FALSE values into 1/0


traindf.loc[:, ("y1")] = traindf.loc[:, ("loan_status")]
traindf.loc[:, ("y2")] = traindf["y1"] == 0           # y2 is the negation of y1
traindf.loc[:, ("y2")] = traindf["y2"].astype(int)    # Turn TRUE/FALSE values into 1/0


validatedf.loc[:, ("y1")] = validatedf.loc[:, ("loan_status")]
validatedf.loc[:, ("y2")] = validatedf["y1"] == 0           # y2 is the negation of y1
validatedf.loc[:, ("y2")] = validatedf["y2"].astype(int)    # Turn TRUE/FALSE values into 1/0

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def min_max_scale_normalization(df):
    scaler = MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
df = dataframe.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim',
                           'y1',
                           'y2']].dropna()


#df = min_max_scale_normalization(df)

inputX = df.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim'
                           ]].as_matrix()

inputY = df.loc[:, ["y1"]].as_matrix()


print(inputX.shape)
print(inputY.shape)
print('\n')

traindf = traindf.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim',
                           'y1',
                           'y2']].dropna()


#traindf = min_max_scale_normalization(traindf)

inputXtrain = traindf.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim'
                           ]].as_matrix()

inputYtrain = traindf.loc[:, ["y1"]].as_matrix()
print(inputXtrain.shape)
print(inputYtrain.shape)
print('\n')


validatedf = validatedf.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim',
                           'y1',
                           'y2']].dropna()

#validatedf = min_max_scale_normalization(validatedf)


inputXvalidate = validatedf.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim'
                           ]].as_matrix()

inputYvalidate = validatedf.loc[:, ["y1"]].as_matrix()

print(inputXvalidate.shape)
print(inputYvalidate.shape)

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(inputXtrain, np.ravel(inputYtrain))

In [ ]:
y_pred = logreg.predict(inputXvalidate)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(inputXvalidate, inputYvalidate)))

In [ ]:
import datetime
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
timeStart = datetime.datetime.now()
kfold = model_selection.KFold(n_splits=100, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, inputXtrain, np.ravel(inputYtrain), cv=kfold, scoring=scoring)
timeEnd = datetime.datetime.now()
executionTime = timeEnd - timeStart
print("Executon Time ", executionTime  )
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(inputYvalidate, logreg.predict(inputXvalidate))
fpr, tpr, thresholds = roc_curve(inputYvalidate, logreg.predict_proba(inputXvalidate)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()